<a href="https://colab.research.google.com/github/bonillahermes/Data_Science_Projects/blob/main/EDA9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hermes Yate Bonilla
**Data Scientist**
---

**Contact:**
- **Email:** [bonillahermes@gmail.com](mailto:bonillahermes@gmail.com)
- **LinkedIn:** [linkedin.com/in/bonillahermes](https://www.linkedin.com/in/bonillahermes/)
- **GitHub:** [github.com/bonillahermes](https://github.com/bonillahermes)
- **Webpage:** [bonillahermes.com](https://bonillahermes.com/)
---

# Análisis de la Relación entre Ozono y Factores Ambientales en Bogotá

## Introducción

El estudio de la calidad del aire es un área de interés creciente debido a su impacto directo en la salud pública y el medio ambiente. Entre los diversos contaminantes presentes en el aire, el ozono (O3) destaca por su doble papel: mientras que en la estratósfera protege la vida en la Tierra al filtrar la radiación ultravioleta, a nivel del suelo es un contaminante nocivo, resultado de reacciones químicas entre óxidos de nitrógeno (NOx) y compuestos orgánicos volátiles (COV) en presencia de luz solar.

## Objetivos

1. **Objetivo Principal**: Analizar la relación entre los niveles de ozono en Bogotá y factores como la temperatura, la radiación solar y los niveles de óxidos de nitrógeno (NOx), utilizando técnicas estadísticas y de modelado de datos.
   
2. **Objetivos Secundarios**:
   - Realizar un análisis estadístico descriptivo para comprender las características de los datos.
   - Investigar la correlación entre el ozono y las variables mencionadas.
   - Desarrollar un modelo lineal para predecir los niveles de ozono en función de estas variables.

In [ ]:
# pip install fancyimpute
#!pip install fancyimpute

## Librerías Necesarias

In [ ]:
import pandas as pd
from fancyimpute import IterativeImputer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd

## Cargar base de datos

In [ ]:
# URL del archivo Excel en el repositorio de GitHub
url = "https://github.com/bonillahermes/Datasets/blob/main/calair.xlsx?raw=true"

# Cargar los datos en un DataFrame de pandas
df = pd.read_excel(url)

## Características de la base

### Descripción de las Variables del Conjunto de Datos de Calidad del Aire

La base de datos proporcionada contiene registros sobre la calidad del aire en Bogotá desde enero de 2021 hasta diciembre de 2023. El conjunto de datos consta de 26280 registros y 15 variables, pero después de la depuración solo quedan 14.Cada registro incluye la fecha y hora de la medición. Las siguientes variables, cada una representando un aspecto específico de la calidad del aire y las condiciones ambientales:

- **PM10 (float64)**: Partículas en suspensión con un diámetro de 10 micrómetros o menos. Estas partículas son capaces de penetrar en los pulmones y pueden afectar la salud respiratoria.

- **PM2.5 (float64)**: Partículas en suspensión con un diámetro de 2.5 micrómetros o menos. Estas partículas finas pueden penetrar profundamente en el sistema respiratorio, presentando riesgos para la salud.

- **NO (float64)**: Concentración de óxido nítrico en el aire, un gas que puede afectar la calidad del aire y la salud.

- **NO2 (float64)**: Concentración de dióxido de nitrógeno, un gas nocivo producido por emisiones vehiculares y otras fuentes de combustión.

- **NOX (float64)**: Concentración total de óxidos de nitrógeno (NO y NO2 combinados), que contribuyen a la contaminación del aire y tienen efectos sobre la salud humana.

- **SO2 (float64)**: Concentración de dióxido de azufre en el aire, un gas producido por la quema de combustibles fósiles que contiene azufre.

- **CO (float64)**: Concentración de monóxido de carbono, un gas tóxico producido por la combustión incompleta de combustibles.

- **OZONO (float64)**: Niveles de ozono, un gas que en la troposfera es un contaminante dañino, pero en la estratósfera forma una capa protectora contra la radiación ultravioleta.

- **Vel_Viento (float64)**: Velocidad del viento en kilómetros por hora, un factor importante en la dispersión de contaminantes atmosféricos.

- **Dir_Viento (float64)**: Dirección del viento en grados, crucial para entender la dispersión de los contaminantes en el aire.

- **Temperatura (float64)**: Temperatura ambiental en grados Celsius, que puede influir en la formación y dispersión de contaminantes.

- **Presion_Baro (float64)**: Presión barométrica en milibares, que puede afectar la calidad del aire y la dispersión de los contaminantes.

- **Rad_Solar (float64)**: Radiación solar recibida en watts por metro cuadrado, crucial en la formación de ozono y otros fotoquímicos.


In [ ]:
# Mostrar información básica sobre el DataFrame
df.info()

In [ ]:
# Mostrar las primeras filas del DataFrame para obtener una vista previa de los datos
df.head()

In [ ]:
# Dimensión
df.shape

## Procesamiento de los valores nulos

In [ ]:
# Valores nulos
valores_nulos = df.isnull().sum()

# Porcentaje de valores nulos
valores_nulos_porcentaje = ((valores_nulos / len(df)) * 100).round(2)

# Resultado
print(valores_nulos_porcentaje)


### Estadísticos Descriptivos Iniciales



- **PM10**: Las partículas PM10 tienen un promedio de 39.79 µg/m³ y una desviación estándar de 25.05 µg/m³, lo que indica una variabilidad significativa en la concentración de estas partículas en el aire de Bogotá. El rango de 0 a 197.8 µg/m³ sugiere episodios de alta contaminación.

- **PM2.5**: Estas partículas más finas presentan un promedio de 18.16 µg/m³ y una desviación estándar de 12.23 µg/m³, lo que también señala una variabilidad considerable. El valor máximo de 137.5 µg/m³ indica posibles eventos de contaminación aguda.

- **NO, NO2, NOX**: Los óxidos de nitrógeno registran promedios de 14.95 µg/m³, 15.38 µg/m³ y 30.34 µg/m³ respectivamente, con desviaciones estándar que reflejan variabilidad en las concentraciones. Los valores máximos significativos sugieren la influencia de fuentes de emisión como el tráfico vehicular y actividades industriales.

- **SO2**: El dióxido de azufre tiene un promedio más bajo de 2.84 µg/m³ con una desviación estándar de 2.25 µg/m³, y un máximo de 37.4 µg/m³, indicando una menor fluctuación en su presencia en el aire de la ciudad.

- **CO**: El monóxido de carbono presenta un promedio de 0.60 µg/m³ y una desviación estándar mínima, con un máximo de solo 6.3 µg/m³, lo que indica niveles generalmente bajos y una menor preocupación en términos de contaminación por CO.

- **OZONO**: Con un promedio de 13.14 µg/m³ y una desviación estándar de 10.53 µg/m³, el ozono muestra una dispersión significativa, probablemente reflejando la complejidad de sus patrones de formación. La mediana de 19.4 µg/m³ sugiere que más de la mitad de las mediciones se mantienen por debajo de este valor.

- **Vel_Viento y Dir_Viento**: La velocidad del viento tiene un promedio de 2.45 m/s y una desviación estándar de 1.32 m/s, indicando variabilidad moderada, con ráfagas que alcanzan un máximo de 8.6 m/s.

- **Temperatura**: La temperatura promedio es de 14.25°C con una desviación estándar de 2.28°C, indicando un clima relativamente estable, lo cual es relevante ya que la temperatura puede influir en las reacciones químicas que involucran contaminantes y en la formación de ozono.

- **Presion_Baro**: La presión barométrica muestra un promedio de 557.21 milibares con una desviación estándar de 1.07 milibares, lo cual es típico para la medición de este parámetro en una ubicación geográfica específica.

- **Rad_Solar**: La radiación solar tiene un promedio de 179.43 W/m² y una desviación estándar significativa de 262.18 W/m², reflejando la gran variabilidad asociada con el ciclo diurno y las condiciones climáticas. Los valores máximos indican la presencia de días muy soleados que son propicios para la formación de ozono.


In [ ]:
df_stats = df.describe()
df_stats = df_stats.transpose()
df_stats

In [ ]:
# Eliminar los valores nulos de la columna 'OZONO'
df = df.dropna(subset=['OZONO'])

In [ ]:
# Valores nulos
valores_nulos = df.isnull().sum()

# Porcentaje de valores nulos
valores_nulos_porcentaje = ((valores_nulos / len(df)) * 100).round(2)

# Resultado
print(valores_nulos_porcentaje)

## Eliminar variables

In [ ]:
df.drop('Precipitacion', axis=1, inplace=True)
df.info()

In [ ]:
df.shape

In [ ]:
# Separar la columna 'DateTime' y las columnas numéricas
df_datetime = df[['DateTime']]
df_numeric = df.drop('DateTime', axis=1)

# Crear histogramas para todas las columnas numéricas
#for column in df_numeric.columns:
    #plt.figure(figsize=(8, 4))
    #sns.histplot(df_numeric[column], kde=True)
    #plt.title(f'Histograma de {column}')
    #plt.xlabel(column)
    #plt.ylabel('Frecuencia')
    #plt.show()

In [ ]:
df_numeric.shape

## Estadísticas Descriptivas Después de la limpieza

- **PM10**: Promedio de 40.31 µg/m³ y desviación estándar de 24.95 µg/m³, indicando variabilidad significativa y posibles episodios de alta contaminación (máximo de 197.8 µg/m³).

- **PM2.5**: Promedio de 18.52 µg/m³ y desviación estándar de 12.26 µg/m³, con un máximo de 137.5 µg/m³, reflejando la presencia de partículas finas que pueden penetrar profundamente en los pulmones.

- **NO, NO2, NOX**: Promedios de 15.15 µg/m³, 15.63 µg/m³ y 30.77 µg/m³ respectivamente, con desviaciones estándar que muestran variabilidad y máximos significativos (233.3 µg/m³ para NO y 264.9 µg/m³ para NOX), sugiriendo influencias de fuentes intensas de emisión como el tráfico y la industria.

- **SO2**: Un promedio más bajo de 2.88 µg/m³ y desviación estándar de 2.26 µg/m³, con un máximo de 37.4 µg/m³, indicando una menor fluctuación en su presencia.

- **CO**: Promedio muy bajo de 0.61 µg/m³ y desviación estándar mínima (0.51 µg/m³), con un máximo de solo 6.3 µg/m³, lo que indica niveles bajos de CO en el aire exterior.

- **OZONO**: Promedio de 13.14 µg/m³ y desviación estándar de 10.53 µg/m³, mostrando dispersión significativa y una mediana de 19.4 µg/m³ que sugiere que más de la mitad de las mediciones están por debajo de este valor.

- **Vel_Viento**: Promedio de 2.42 m/s y desviación estándar de 1.32 m/s, con un máximo de 8.6 m/s, indicando variabilidad en la velocidad del viento que es importante para la dispersión de contaminantes.

- **Dir_Viento**: Variabilidad completa en la dirección del viento con valores de 0 a 360 grados, relevante para la modelización de la dispersión de contaminantes.

- **Temperatura**: Promedio de 14.19°C y desviación estándar de 2.26°C, reflejando un clima templado.

- **Presion_Baro**: Promedio de 557.15 milibares y desviación estándar de 1.04 milibares, indicando condiciones atmosféricas estables.

- **Rad_Solar**: Promedio alto de 174.89 W/m² y una gran desviación estándar de 260.22 W/m², lo que refleja una variabilidad significativa debido a factores como la nubosidad y la hora del día. El valor máximo de 1237 W/m² indica días con una intensa radiación solar.


In [ ]:
df_stats = df.describe()
df_stats = df_stats.transpose()
df_stats

## Imputación de datos

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Configurar los parámetros del imputador
mice_imputer = IterativeImputer(
    max_iter=50,
    tol=0.001,
    imputation_order='ascending',
    initial_strategy='median',
    add_indicator=False,
    min_value=None,
    max_value=None,
    random_state=42
)

# Aplicar MICE a las columnas numéricas
df_numeric_imputed = mice_imputer.fit_transform(df_numeric)

In [ ]:
# Convertir el resultado a DataFrame
df_numeric_imputed = pd.DataFrame(df_numeric_imputed, columns=df_numeric.columns)

In [ ]:
df_numeric_imputed.shape

In [ ]:
# Calcular el porcentaje de valores negativos por cada variable
porcentaje_negativos_por_variable = ((df_numeric_imputed < 0).sum() / df_numeric_imputed.count() * 100).round(2)
print(porcentaje_negativos_por_variable)


In [ ]:
# Calcular la mediana de cada columna
medianas = df_numeric_imputed.median()

# Reemplazar los valores negativos con la mediana correspondiente de cada columna
df_numeric_imputed = df_numeric_imputed.apply(lambda x: np.where(x < 0, medianas[x.name], x))
df_numeric_imputed.shape


In [ ]:
# Calcular el porcentaje de valores negativos por cada variable
porcentaje_negativos_por_variable = ((df_numeric_imputed < 0).sum() / df_numeric_imputed.count() * 100).round(2)
print(porcentaje_negativos_por_variable)

In [ ]:
df_datetime.shape

In [ ]:
# Restablecer el índice de df_datetime
df_datetime = df_datetime.reset_index(drop=True)

# Reintegrar la columna 'DateTime'
df = pd.concat([df_datetime, df_numeric_imputed], axis=1)

df.shape

In [ ]:
# Valores nulos
valores_nulos = df.isnull().sum()

# Porcentaje de valores nulos
valores_nulos_porcentaje = ((valores_nulos / len(df)) * 100).round(2)

# Resultado
print(valores_nulos_porcentaje)

## Estadísticos Descriptivos con Imputación de datos

- **PM10**: La concentración media de 40.72 µg/m³ y la desviación estándar de 24.98 µg/m³ indican una alta variabilidad en las mediciones de PM10, con valores que oscilan entre 0 y 197.8 µg/m³, apuntando a posibles eventos de contaminación significativos.

- **PM2.5**: Con un promedio de 18.50 µg/m³ y una desviación estándar de 12.23 µg/m³, las partículas PM2.5 también muestran una considerable variabilidad y la capacidad de alcanzar niveles preocupantes (máximo de 137.5 µg/m³).

- **NO, NO2, NOX**: Los óxidos de nitrógeno presentan promedios de 15.06 µg/m³, 15.62 µg/m³ y 30.68 µg/m³ respectivamente, con máximos de hasta 233.3 µg/m³ para NO y 264.9 µg/m³ para NOX, lo que revela episodios de alta concentración de estos contaminantes.

- **SO2**: El dióxido de azufre muestra una concentración media más baja (2.88 µg/m³) y una desviación estándar reducida (2.26 µg/m³), indicando niveles más consistentes en comparación con los NOx.

- **CO**: Con un promedio de 0.62 µg/m³ y una desviación estándar de 0.51 µg/m³, el CO tiene niveles generalmente bajos, con un máximo reportado de 6.3 µg/m³, indicativo de una menor preocupación por este gas en el ambiente.

- **OZONO**: El ozono tiene una media de 13.14 µg/m³ y una desviación estándar de 10.54 µg/m³, con un máximo de 69 µg/m³. La variabilidad y el rango de concentraciones sugieren una influencia significativa de factores ambientales en su formación.

- **Vel_Viento**: La velocidad promedio del viento es de 2.41 m/s, con una desviación estándar de 1.32 m/s y un máximo de 8.6 m/s. Estos valores indican que, en general, el viento en Bogotá podría no ser lo suficientemente fuerte como para dispersar efectivamente los contaminantes, lo cual es relevante para las estrategias de gestión de calidad del aire.

- **Dir_Viento**: La dirección del viento promedia 193.90 grados con una desviación estándar amplia de 84.06 grados, lo que indica una variabilidad significativa en la dirección desde la cual el viento sopla.

- **Temperatura**: La temperatura media de 14.20°C y una desviación estándar de 2.26°C muestra un clima moderado, lo cual es crucial para entender las reacciones químicas que afectan la calidad del aire y la formación de ozono.

- **Presion_Baro**: La presión barométrica promedio de 557.16 milibares con una desviación estándar mínima de 1.03 milibares sugiere que hay poca variación en la presión atmosférica, lo que es importante para estudiar los patrones de dispersión de contaminantes.

- **Rad_Solar**: Un promedio de 172.40 W/m² y una desviación estándar de 241.32 W/m² reflejan la alta variabilidad en la radiación solar recibida, lo cual es un factor determinante en la formación de ozono y en la dinámica de otros contaminantes fotoquímicos.


In [ ]:
df_stats = df.describe()
df_stats = df_stats.transpose().round(2)
df_stats

In [ ]:
# Separar la columna 'DateTime' y las columnas numéricas
df_numeric = df.drop('DateTime', axis=1)

# Crear histogramas para todas las columnas numéricas utilizando Seaborn
for column in df_numeric.columns:
    plt.figure(figsize=(8, 4))
    sns.histplot(df_numeric[column], kde=True)
    plt.title(f'Histograma de {column}')
    plt.xlabel(column)
    plt.ylabel('Frecuencia')
    plt.show()

In [ ]:
# Crear boxplots para todas las columnas numéricas utilizando Seaborn
for column in df_numeric.columns:
    plt.figure(figsize=(8, 4))
    sns.boxplot(x=df_numeric[column])
    plt.title(f'Boxplot de {column}')
    plt.xlabel(column)
    plt.show()

## Modelo 1: Análisis de correlación

Los resultados de las correlaciones significativas muestran que existen relaciones significativas entre las siguientes variables:

* **PM10 y PM2.5:** **(0.8578, +)** Las partículas en suspensión (PM) son un contaminante común del aire, y las partículas de menor tamaño, como las PM2.5, son las más perjudiciales para la salud. La correlación positiva entre estas dos variables indica que las concentraciones de PM10 y PM2.5 suelen aumentar y disminuir juntas.
* **NO y NOX:** **(0.9644, +)** El óxido nítrico (NO) es un precursor del dióxido de nitrógeno (NO2), otro contaminante del aire. La correlación positiva entre estas dos variables indica que las concentraciones de NO y NO2 suelen aumentar y disminuir juntas.
* **NO y CO:** **(0.8737, +)** El monóxido de carbono (CO) es otro contaminante del aire que se produce por la combustión incompleta de combustibles. La correlación positiva entre NO y CO indica que las concentraciones de estos dos contaminantes suelen aumentar y disminuir juntas.
* **NO2 y NOX:** **(0.7859, +)** El NO2 es un precursor del ozono (O3), otro contaminante del aire. La correlación positiva entre estas dos variables indica que las concentraciones de NO2 y O3 suelen aumentar y disminuir juntas.
* **NO2 y CO:** **(0.7478, +)** El CO es un contaminante del aire que puede reaccionar con el NO2 para formar O3. La correlación positiva entre NO2 y CO indica que las concentraciones de estos dos contaminantes suelen aumentar y disminuir juntas.
* **NOX y CO:** **(0.9178, +)** El CO es un contaminante del aire que puede reaccionar con los óxidos de nitrógeno (NOX) para formar O3. La correlación positiva entre NOX y CO indica que las concentraciones de estos dos contaminantes suelen aumentar y disminuir juntas.
* **O3 y viento:** **(-0.6204, -)** El viento puede ayudar a dispersar el ozono, por lo que la correlación negativa entre estas dos variables indica que las concentraciones de O3 suelen disminuir cuando aumenta la velocidad del viento.
* **O3 y temperatura:** **(0.6488, +)** La temperatura puede influir en la formación de ozono, por lo que la correlación positiva entre estas dos variables indica que las concentraciones de O3 suelen aumentar cuando aumenta la temperatura.
* **Viento y temperatura:** **(0.6383, +)** La temperatura puede influir en la velocidad del viento, por lo que la correlación positiva entre estas dos variables indica que la velocidad del viento suele aumentar cuando aumenta la temperatura.

Estos resultados son consistentes con los conocimientos existentes sobre la calidad del aire. Por ejemplo, se sabe que las PM10 y PM2.5 suelen aumentar y disminuir juntas porque se forman a partir de las mismas fuentes, como la combustión de combustibles fósiles y la actividad industrial. También se sabe que los óxidos de nitrógeno (NOX) y el ozono (O3) son precursores, lo que significa que se forman a partir de otros contaminantes del aire.


In [ ]:
corr_matrix = df_numeric.corr()
corr_matrix

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, square=True)
plt.title('Matriz de Correlación para Variables Numéricas')
plt.show()

In [ ]:
# Calcular las correlaciones
correlations = df_numeric.corr(method="pearson")

# Definir el nivel de significancia
significancia = 0.05

# Encontrar las correlaciones significativas
significant_correlations = []
for i in range(len(correlations)):
    for j in range(i + 1, len(correlations)):
        if abs(correlations.iloc[i, j]) >= significancia and (correlations.iloc[i, j] >= 0.6 or correlations.iloc[i, j] <= -0.6):
            significant_correlations.append((correlations.iloc[i, j], df_numeric.columns[i], df_numeric.columns[j]))

# Imprimir las correlaciones significativas
print(significant_correlations)

In [ ]:
sns.pairplot(df_numeric[['PM10', 'PM2.5', 'NO', 'NOX', 'CO', 'NO2', 'OZONO', 'Vel_Viento','Temperatura', 'Rad_Solar']], diag_kind="kde")

## Modelo 2: Análisis de Regresión

### Modelo general

* El modelo de regresión lineal explica el **58,5%** de la varianza de la concentración de ozono (R-cuadrado ajustado). Esto significa que el modelo es moderadamente bueno para predecir los niveles de ozono en función de las variables independientes seleccionadas.
* El valor de F-estadístico es alto y significativo (F(11, 23532) = 3021, p < 0.001), lo que indica que el modelo globalmente es significativo.

### Interpretación de las variables individuales

* **PM10 y PM2.5:** Ambas partículas finas tienen un efecto **positivo** y **significativo** sobre la concentración de ozono. Un aumento de 1 unidad en PM10 se asocia con un aumento de **0,022 unidades** en ozono, y un aumento de 1 unidad en PM2.5 se asocia con un aumento de **0,15 unidades** en ozono.
* **NO, NO2 y NOX:** Estos óxidos de nitrógeno no tienen un efecto **estadísticamente significativo** sobre la concentración de ozono. Sin embargo, los coeficientes negativos sugieren una posible relación **no lineal**, que podría explorarse con transformaciones de las variables o modelos más complejos.
* **SO2:** El dióxido de azufre tiene un efecto **negativo** y **significativo** sobre la concentración de ozono. Un aumento de 1 unidad en SO2 se asocia con una disminución de **0,06 unidades** en ozono.
* **CO:** El monóxido de carbono tiene un efecto **positivo** y **significativo** sobre la concentración de ozono. Un aumento de 1 unidad en CO se asocia con un aumento de **3,28 unidades** en ozono.
* **Vel_Viento:** La velocidad del viento tiene un efecto **positivo** y **significativo** sobre la concentración de ozono. Un aumento de 1 unidad en la velocidad del viento se asocia con un aumento de **1,51 unidades** en ozono, lo que sugiere que el viento ayuda a dispersar el ozono.
* **Temperatura:** La temperatura tiene un efecto **positivo** y **significativo** sobre la concentración de ozono. Un aumento de 1 grado Celsius se asocia con un aumento de **0,76 unidades** en ozono.
* **Presion_Baro y Rad_Solar:** La presión barométrica y la radiación solar no tienen un efecto **significativo** sobre la concentración de ozono.

### Advertencias

* El número de condición del modelo es alto, lo que sugiere que podría haber **multicolinealidad** entre las variables independientes. Esto puede afectar la precisión de las estimaciones de los coeficientes y su estabilidad.
* La prueba de normalidad de los residuos indica que no siguen una distribución **normal**. Esto podría afectar la validez de las inferencias basadas en el modelo.

### Consideraciones adicionales

* Este modelo es un primer paso para entender la relación entre los contaminantes del aire y la concentración de ozono en Bogotá. Se podrían explorar modelos más complejos o técnicas de análisis de datos para obtener una mayor comprensión de las relaciones no lineales y las interacciones entre variables.
* Es importante tener en cuenta que este modelo solo explica una parte de la varianza de la concentración de ozono. Otros factores no incluidos en el modelo, como la topografía de la ciudad, las fuentes de emisión no puntuales y los patrones meteorológicos a gran escala, también pueden influir en los niveles de ozono.



In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

# Seleccionar las variables
X = df[['PM10', 'PM2.5', 'NO', 'NO2', 'NOX', 'SO2', 'CO', 'Vel_Viento', 'Temperatura', 'Presion_Baro', 'Rad_Solar']]
y = df['OZONO']

# Añade una columna de constantes para el intercepto
X = sm.add_constant(X)

# Entrena el modelo con statsmodels
model = sm.OLS(y, X).fit()

# Imprime el resumen del modelo
print(model.summary())


# Guardar base de datos

In [ ]:
# Guarda la base de datos en formato CSV
df.to_csv("base_calidad_aire_bogota.csv", index=False)


# Comportamiento del Ozono a través del tiempo

In [ ]:
# Obtiene la concentración de ozono
ozono = df['OZONO']

# Obtiene la fecha y hora
fecha_hora = df['DateTime']

# Grafica el comportamiento del ozono
plt.plot(fecha_hora, ozono)

# Añade un título al gráfico
plt.title('Concentración de ozono en Bogotá')

# Añade una etiqueta al eje x
plt.xlabel('Fecha y hora')

# Añade una etiqueta al eje y
plt.ylabel('Concentración (ppb)')

# Muestra el gráfico
plt.show()